In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
import tensorflow_datasets as tfds
import numpy as np
import os



## Use Cats & Dogs Dataset Load a subset of the data
##Create random bounding box in each image
## Define a model that is able to do both:
## 1. Classify the image either cat or dog
## 2. Predict the bounding box that was generated in the begging

### setup

In [16]:


# path to images
data_dir = "data/train/"

img_size = 128

images = []# hold image tensors after loading
labels = [] # hold corresponding labels for each list

for fname in os.listdir(data_dir):
    if fname.endswith(".jpg"):
        img_path = os.path.join(data_dir, fname)
        #load and resize image
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, (img_size, img_size))
        img = tf.cast(img, tf.float32)

        images.append(img)

        # assing label based on filename
        if "cat" in fname.lower():
            labels.append(0)
        elif "dog" in fname.lower():
            labels.append(1)
        else:
            continue # skips it if not clear


# converts to tensors

images = tf.stack(images)
labels = tf.convert_to_tensor(labels, dtype=tf.int32)







### Create a random bounding box in each image

In [17]:
#to create t random bounding box you need to pick two random x and y and it starts with minium x and then starts with min y then max x nad max y

def preprocess_bound_box(image):
    #resizes and normalize the image
    image = tf.image.resize(image, (img_size, img_size))
    image = tf.cast(image, tf.float32) / 255.0 # divide by 255.0 to normalize pixel values
    #pixel values range 0,255 for each color channel
    # Nueral network work better with smaller ranges

    #Generate random bounding box in nroamlized coordinates
    #Ensure   x_min < x_max and y_min < y_max
#picks random float in the range
    x_min = np.random.uniform(0, 0.5) #0, 0.5 to make sure box starts somewhree top left half
    y_min = np.random.uniform(0, 0.5)

    x_max = np.random.uniform(0.5, 1.0)
    y_max = np.random.uniform(0.5, 1.0)

    boundbox = np.array([x_min, y_min, x_max, y_max], dtype=np.float32)

    return image, boundbox

#ex usage with list of images

processed_images = []
boundboxes = []

for img in images:
    img_proc, boundbox = preprocess_bound_box(img)
    processed_images.append(img_proc)
    boundboxes.append(boundbox)

processed_images = tf.stack(processed_images)
boundboxes = tf.stack(boundboxes)

print("Processed images shape", processed_images.shape)
print("Boundboxes shape", boundboxes.shape)
print("Example Bound box:", boundboxes[0].numpy())





Processed images shape (4097, 128, 128, 3)
Boundboxes shape (4097, 4)
Example Bound box: [0.43754366 0.26004314 0.65264034 0.5635852 ]
